In [1]:
#!pip install google-cloud-bigquery
#!pip install mlxtend
#!pip install plotly
from google.cloud import bigquery
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
import plotly as px

In [4]:
# Configuración del cliente de BigQuery
client = bigquery.Client()

# Definición de la consulta SQL
query = """
    SELECT * FROM `vanti-poc-440213.data_poc.raw_customers`
"""

# Ejecuta la consulta y guarda los resultados en un DataFrame
try:
    customers = client.query(query).to_dataframe()
    print("Consulta ejecutada exitosamente. Aquí tienes las primeras filas:")
    print(df.head())
except Exception as e:
    print(f"Error al ejecutar la consulta: {e}")

OSError: Project was not passed and could not be determined from the environment.

In [4]:
customers

,int64_field_0,IdCustomer,Customer,Address,Telephone,Nit,Service,State,IdCompany,ContactNamePayment,...,Documents,Observation,NDays,CreateDate,UpgradeDate,UserCreate,UserUpdate,CalendarRate,Periodicity,IdCountry
0,8,1107,TALENTO & CULTURA PERIFERIA,Ak 19 No. 95- 12/20,6000080,900310011-9,Selección de personal,0,2,X,...,x,XX,x,2021-02-25,NaT,psyuly1986@gmail.com,None,0,0.0,1
1,9,1106,TALENTO & CULTURA CBIT,Ak 19 No. 95- 12/20,6000080,900.087.315-3,Selección de personal,0,2,X,...,x,x,x,2021-02-25,NaT,psyuly1986@gmail.com,None,0,0.0,1
2,74,1117,FABRICA PORVENIR,Ak 19 No. 95- 12/20,6000080,900.087.315-3,Fabrica para Porvenir,1,2,X,...,x,x,x,2021-03-04,NaT,psyuly1986@gmail.com,None,0,3.0,1
3,79,1115,FABRICA BANCO POPULAR,Ak 19 No. 95- 12/20,6000080,900.087.315-3,Servicios de fabrica Banco Popular,1,2,X,...,x,x,x,2021-03-04,NaT,psyuly1986@gmail.com,None,0,3.0,1
4,80,1113,FABRICA BANCO DE OCCIDENTE,Ak 19 No. 95- 12/20,6000080,900.087.315-3,Fabrica para Banco de occidente,1,2,X,...,x,x,x,2021-03-04,NaT,psyuly1986@gmail.com,None,0,3.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,139,69,BANCO COLPATRIA MULTIBANCA COLPATRIA S.A.,"Cra. 7 #24-51, Bogotá",7456300,860034594,"IT Staffing, Fabrica",1,1,ALEJANDRO SOTOMAYOR,...,na,na,45,2017-06-12,2024-02-05,carolina.alvarez@periferiaitgroup.com,mila.a.per@gmail.com,1,3.0,1
155,47,1097,LIBERTY SEGUROS SA,Calle 72 #10-07,7057050,860039988,SEGUROS,1,1,FREDY FONSECA,...,Contrato,.,30 dias,2019-09-30,2021-07-16,nathalie.barrios@periferiaitgroup.com,nathalie.barrios@periferiaitgroup.com,1,2.0,1
156,64,1126,FUNDACION SANTA FE DE BOGOTÁ,Calle 119 #7-75 -Piso 2,6030303,860037950,Nearshore,1,2,OSCAR GOMEZ,...,ninguna,None,60,2021-09-10,2021-09-10,nathalie.barrios@periferiaitgroup.com,andresleonqu@gmail.com,1,3.0,1
157,119,1090,BYTTE,Calle 26 #69 D-91 Torre 1 Of 407,7443800,830115764-4,CONSULTORIA,1,2,Paola Pardo,...,Contrato,CONTRATO,30 días,2019-06-20,2019-06-20,nathalie.barrios@periferiaitgroup.com,nathalie.barrios@periferiaitgroup.com,1,3.0,1


In [7]:
# Paso 1: Importar bibliotecas
import pandas as pd
import jellyfish
from itertools import combinations

# Paso 2: Cargar y preparar datos
#df = pd.read_csv('gs://ruta-a-tu-bucket/clientes.csv')
customers['nombre_normalizado'] = customers['Customer'].str.lower().str.strip()

# Paso 3: Definir función de similitud
def calcular_similitud_jaro_winkler(cadena1, cadena2):
    return jellyfish.jaro_winkler_similarity(cadena1, cadena2)

# Paso 4: Implementar bloqueo
customers['bloque'] = customers['nombre_normalizado'].str[0]

# Paso 5: Encontrar posibles duplicados
posibles_duplicados = []

for bloque, grupo in customers.groupby('bloque'):
    registros = grupo['nombre_normalizado'].tolist()
    indices = grupo.index.tolist()
    
    for i in range(len(registros)):
        for j in range(i+1, len(registros)):
            nombre1 = registros[i]
            nombre2 = registros[j]
            idx1 = indices[i]
            idx2 = indices[j]
            similitud = calcular_similitud_jaro_winkler(nombre1, nombre2)
            if similitud >= 0.90:
                posibles_duplicados.append((idx1, idx2, similitud))

# Paso 6: Mostrar resultados
df_duplicados = pd.DataFrame(posibles_duplicados, columns=['idx1', 'idx2', 'similitud'])
df_duplicados['nombre1'] = customers.loc[df_duplicados['idx1'], 'Customer'].values
df_duplicados['nombre2'] = customers.loc[df_duplicados['idx2'], 'Customer'].values

print(df_duplicados[['nombre1', 'nombre2', 'similitud']])


                          nombre1                    nombre2  similitud
0   AREA ADMINISTRATIVA PERIFERIA  AREA  ADMINISTRATIVA CBIT   0.912828
1  BANCO DE BOGOTÁ (PÁNAMA), S.A.            BANCO DE BOGOTA   0.900000
2      FABRICA BANCO DE OCCIDENTE    FABRICA BANCO DE BOGOTA   0.911371
3               FABRICA AV VILLAS               FABRICA AVAL   0.924510
4                     FABRICA ATH                 FABRICA QA   0.943636
5                     FABRICA ATH               FABRICA AVAL   0.913636
6                      FABRICA QA               FABRICA AVAL   0.930000
7        FINANCIERA EFECTIVA PERÚ   FINANCIERA EFECTIVA PERÚ   1.000000
8             SURA - SEGUROS VIDA   SURA - SEGUROS GENERALES   0.901754
9    TALENTO & CULTURA PERIFERIA      TALENTO & CULTURA CBIT   0.913468


In [10]:
# Paso 1: Importar bibliotecas
import pandas as pd
import Levenshtein
from itertools import combinations

# Paso 2: Cargar y preparar datos
# customers = pd.read_csv('gs://ruta-a-tu-bucket/clientes.csv')
customers['nombre_normalizado'] = customers['Customer'].str.lower().str.strip()

# Paso 3: Definir función de similitud
def calcular_similitud_levenshtein(cadena1, cadena2):
    distancia = Levenshtein.distance(cadena1, cadena2)
    longitud_maxima = max(len(cadena1), len(cadena2))
    if longitud_maxima == 0:
        return 1.0  # Ambas cadenas están vacías
    similitud = 1 - (distancia / longitud_maxima)
    return similitud

# Paso 4: Implementar bloqueo
customers['bloque'] = customers['nombre_normalizado'].str[0]

# Paso 5: Encontrar posibles duplicados
posibles_duplicados = []

for bloque, grupo in customers.groupby('bloque'):
    registros = grupo['nombre_normalizado'].tolist()
    indices = grupo.index.tolist()
    
    for i in range(len(registros)):
        for j in range(i+1, len(registros)):
            nombre1 = registros[i]
            nombre2 = registros[j]
            idx1 = indices[i]
            idx2 = indices[j]
            similitud = calcular_similitud_levenshtein(nombre1, nombre2)
            if similitud >= 0.90:
                posibles_duplicados.append((idx1, idx2, similitud))

# Paso 6: Mostrar resultados
df_duplicados = pd.DataFrame(posibles_duplicados, columns=['idx1', 'idx2', 'similitud'])
df_duplicados['nombre1'] = customers.loc[df_duplicados['idx1'], 'Customer'].values
df_duplicados['nombre2'] = customers.loc[df_duplicados['idx2'], 'Customer'].values

print(df_duplicados[['nombre1', 'nombre2', 'similitud']])


                    nombre1                   nombre2  similitud
0  FINANCIERA EFECTIVA PERÚ  FINANCIERA EFECTIVA PERÚ        1.0


In [12]:
customers[customers["Customer"]=='FINANCIERA EFECTIVA PERÚ']

,int64_field_0,IdCustomer,Customer,Address,Telephone,Nit,Service,State,IdCompany,ContactNamePayment,...,NDays,CreateDate,UpgradeDate,UserCreate,UserUpdate,CalendarRate,Periodicity,IdCountry,nombre_normalizado,bloque
115,66,1154,FINANCIERA EFECTIVA PERÚ,Av el Derby Nro. 254 Int. 2101 (Edificio Lima ...,51997517082,20441805960,Financiero,1,1,MIGUEL ALARCON,...,30 dias,2023-01-31,NaT,Vivialexandra_11@hotmail.com,None,1,NaN,1,financiera efectiva perú,f
116,67,1160,FINANCIERA EFECTIVA PERÚ,JR CRISTÓBAL DE PERALTA NORTE 820 SURCO,51 997 978 939,20100028698,Distribuye maquinaria pesada y principal prove...,1,1,JUAN JOSE GOMEZ,...,60 dias,2023-03-09,NaT,Vivialexandra_11@hotmail.com,None,1,NaN,1,financiera efectiva perú,f


In [ ]:
# Paso 1: Importar bibliotecas
import pandas as pd
from pyphonetics import Soundex, Metaphone
import unidecode

# Paso 2: Cargar y preparar datos
df = pd.read_csv('gs://ruta-a-tu-bucket/clientes.csv')
df['nombre_normalizado'] = df['nombre'].apply(normalizar_cadena)

# Paso 3: Inicializar codificadores
soundex = Soundex()
metaphone = Metaphone()

# Paso 4: Codificar los nombres
df['soundex'] = df['nombre_normalizado'].apply(soundex.phonetics)
df['metaphone'] = df['nombre_normalizado'].apply(metaphone.phonetics)

# Paso 5: Encontrar posibles duplicados usando Metaphone
posibles_duplicados = []

for codigo_metaphone, grupo in df.groupby('metaphone'):
    if len(grupo) > 1:
        indices = grupo.index.tolist()
        for i in range(len(indices)):
            for j in range(i+1, len(indices)):
                idx1 = indices[i]
                idx2 = indices[j]
                nombre1 = df.loc[idx1, 'nombre_normalizado']
                nombre2 = df.loc[idx2, 'nombre_normalizado']
                posibles_duplicados.append((idx1, idx2, codigo_metaphone))

# Paso 6: Mostrar resultados
df_duplicados = pd.DataFrame(posibles_duplicados, columns=['idx1', 'idx2', 'metaphone'])
df_duplicados['nombre1'] = df.loc[df_duplicados['idx1'], 'nombre'].values
df_duplicados['nombre2'] = df.loc[df_duplicados['idx2'], 'nombre'].values

print(df_duplicados[['nombre1', 'nombre2', 'metaphone']])